In [ ]:
def isSoundtrack(styles):
    if "Soundtrack" in styles:
        return True
    return False

def getStyles(vals):
    retvals = []
    styles = vals.get("Style")
    if styles is None:
        return retvals
    elif isinstance(styles, list):
        retvals = [x[0] for x in styles]

    return retvals

def getArtists(vals):
    retvals = [x[0] for x in vals]
    return retvals

### Get Soundtracks
tmp = df[df['Profile'].apply(getStyles).apply(isSoundtrack) == True]

### Set Artists
df["Artists"] = df['Artist'].apply(getArtists)

In [ ]:
soundtracks = sorted(list(set(tmp["Album"].to_list())))

term = "Walt Disney's "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Motion Picture Sound Track "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Motion Picture Soundtrack "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Original Motion Picture Soundtrack "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Original Sound Track "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Soundtrack Of "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]

terms = []
terms.append(" (Original Cast Sound Track)")
terms.append(" (Original Cast Soundtrack)")
terms.append(" (Music From The Original Motion Picture Sound Track)")
terms.append(" (Music From The Soundtrack)")
terms.append(" (Music from the soundtrack of)")
terms.append(" (Original Motion Picture Soundtrack)")
terms.append(" (Original Sound Track Recording)")
terms.append(" (Original Soundtrack)")
terms.append(" (Original Sound Track Recording)")
terms.append(" (The Original Sound Track Recording)")
terms.append(" (Original Soundtrack)")
terms.append(" (Original Soundtrack Recording)")
terms.append(" (Music From The Motion Picture Sound Track)")
terms.append(" (Music From The Motion Picture Soundtrack)")
terms.append(":  Music From The Original Motion Picture Soundtrack")
terms.append(" - Music From The Original Motion Picture Soundtrack")
terms.append(" (Music From The Motion Picture)")
terms.append(" (An Original Soundtrack Recording)")
terms.append("  (An Original Soundtrack Recording)")
terms.append(" Original Motion Picture Soundtrack")

terms+=['(Original Television Soundtrack Recording)',
 '(Motion Picture Soundtrack)',
 '(The Original Motion Picture Soundtrack)',
 "(The Original Motion Picture Soundtrack)",
 '(Music From The Original Motion Picture Soundtrack)',
 '(Original Motion Picture Soundtrack Album)',
 "(Music From The Original Motion Picture Soundtrack)",
 "(Original TV Soundtrack)",
 "(From The Motion Picture Soundtrack)",
 '(Soundtrack)',
 '(Limited Event Series Soundtrack)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(Music Composed For The Original Motion Picture Soundtrack)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(The Original Soundtrack Recording)',
 '(original Soundtrack From The Movie)',
 '(The Original Movie Soundtrack)',
 '(The Original Soundtrack From The Motion Picture)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(Music From The Original Soundtrack)',
 '(From The Original Motion Picture Soundtrack)'
 ' - Original Soundtrack Recording',
 ' (An Original Sound Track Recording)',
 ' - Music from the soundtrack of the Motion Picture',
 ' Original Soundtrack',
 ' Original Cast Sound Track Album',
 ' - Original Cast Sound Track Album',
 ' (From The Original Sound Track)',
 ' - Music From The Original Motion Picture Sound Track',
 ' (The Original Motion Picture Sound Track)',
 ' - Original Sound Track Recording',
 ': Original Cast Soundtrack',
 ':  Original Cast Soundtrack',
 ' (Original Cast Sound Track Album)']
for term in terms:
    soundtracks = [x.replace(term, "").strip() if x.endswith(term) else x for x in soundtracks]

print(len(soundtracks))
soundtracks = set(soundtracks)
print(len(soundtracks))
soundtracks

In [ ]:
[y for y in [x for x in soundtracks if x.find("Soundtrack") != -1] if y.find("(") != -1]

In [ ]:
def getStyles(vals):
    retvals = []
    if vals is None:
        return retval
    if isinstance(vals, str):
        vals = json.loads(vals)
    if isinstance(vals, dict):
        styles = vals.get("Style")
        if styles is None:
            return retvals
        elif isinstance(styles, list):
            retvals = [x[0] for x in styles]
            
    return retvals

In [ ]:
x = str(vals)
json.loads(x)
#getStyles(vals)

In [3]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-07-31 15:19:07.580976


In [23]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()
col    = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


In [ ]:
vals = ["1166600", "1324400", "1340600", "15800", "1626700", "1691500", "2006300", "211200", "23500", "2360300", "2701000", "29300", "40000", "4300900", "437700", "444000", "4731500", "563200", "6400", "72800", "90800"]

In [ ]:
modVal=0

artistDBDir = disc.getArtistsDBDir()
dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
dbdata  = getFile(dbname)
print("Found {0} artist IDs in {1}".format(len(dbdata), dbname))

for val in vals:
    del dbdata[val]

savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
print("Saving {0} new artist IDs to {1}".format(len(dbdata), savename))
saveJoblib(data=dbdata, filename=savename, compress=True)


In [ ]:
for modVal in range(0,100):
    try:
        arts.assertDBModValExtraData(modVal)
    except:
        pass

In [5]:
vals="""Downloading: https://www.discogs.com/artist/302000-Ballard-MacDonald?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/75910-Stephen-W-Tayler?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/11415-Bob-Clearmountain?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/834222-Paul-Myers-2?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/674125-Ram%C3%83%C2%B3n-Arcusa?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/253127-Fredrik-Nordstr%C3%83%C2%B6m?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/449529-Denny-Purcell?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/273544-Angus-Young?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/1641558-Jacques-Fournier?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/252962-Alfred-Lion?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/244167-Brendan-OBrien?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/27468-Bill-Bottrell?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/382969-Fred-Kevorkian?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/142073-Bernard-Edwards-Nile-Rodgers?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/255681-Bill-Szymczyk?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/332586-Ralph-Bass?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/180298-Tom-Dowd?sort=year%2Casc&limit=500&page=2"""

In [27]:

artistDBDir = disc.getArtistsDBDir()

/artist/67340-Yaki-Da

urls = vals.split('\n')
for url in urls:
    ref = url.split('?')[0]


    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(url)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if False:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)
    
    url = ref.replace("Downloading: ", "")
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
    sleep(1)

/Volumes/Music/Discog/artists/0/302000.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/302000-Ballard-MacDonald
Saving /Volumes/Music/Discog/artists/0/302000.p
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/10/75910.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/75910-Stephen-W-Tayler
Saving /Volumes/Music/Discog/artists/10/75910.p
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/15/11415.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/11415-Bob-Clearmountain
Saving /Volumes/Music/Discog/artists/15/11415.p
  --> This file is 17.8kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/22/834222.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/834222-Paul-Myers-2
Saving /Volumes/Music/Discog/artists/22/834222.p
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/25/674125.p exists.


In [ ]:
for modVal in range(1,100):
    try:
        arts.assertDBModValData(modVal)
    except:
        pass

In [ ]:
for modVal in [1]:
    start, cmt = clock("Processing {0}".format(modVal))
    arts.parseArtistModValFiles(modVal, force=True)
    elapsed(start, cmt)

In [ ]:
data = getFile("/Volumes/Music/Discog/artists-db/1-DB.p")

In [ ]:
artistIDtoRefData = disc.getArtistIDToRefData()

In [ ]:
for modVal in range(20,60):
    start, cmt = clock("Processing {0}".format(modVal))
    arts.parseArtistModValFiles(modVal, force=True)
    elapsed(start, cmt)

In [ ]:
x = str(artistData.url.url)

In [ ]:
x.Contains("x")

In [ ]:
errs=getFile("errs.p")
print("Found {0} err files".format(len(errs)))
from collections import Counter
n = 0
for item in Counter(refCounts).most_common():
    n += 1
    ref      = item[0]
    cnts     = item[1]
    if errs.get(ref):
        continue
    discID   = arts.discogsUtils.getArtistID(ref)
    url      = arts.getArtistRef(ref)
    savename = arts.getArtistSavename(discID)
    if isFile(savename):
        continue
    try:
        arts.downloadArtistURL(url, savename)
        sleep(1)
    except:
        print("Adding {0} to errs".format(ref))
        errs[ref] = True
        if len(errs) % 10 == 0:
            saveFile(idata=errs, ifile="errs.p", debug=True)
        pass
    
    print("----------> {0} \t {1}".format(cnts,n)
    print("")



In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()        

def f(a_list):
    out = None
    for n in a_list:
        dbname = setFile(artistDBDir, "{0}-DB.p".format(i))            
        dbdata = getFile(dbname, version=3)
        return dbdata
        if out is None:
            out = dbdata
        else:
            out.update(dbdata)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    print(chunks)
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
    print(result)
 
    while not result.ready():
        print("Running...")
        time.sleep(5)
 
    return result.get()
    #return sum(result.get())
    
f_amp(files[:5])

In [ ]:
f_amp(range(100))

In [ ]:
from searchUtils import findExt

artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()        

totalSaves = 0
for i in range(maxModVal):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')
    break

    #dbname = setFile(artistDBDir, "{0}-DB.p".format(i))            
    #dbdata = getFile(dbname, version=3)

In [ ]:
len(files)

In [ ]:
chunks = [files[:10][i::5] for i in range(5)]

In [ ]:
[len(x) for x in chunks]

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()
col = collections(disc)

In [ ]:
from time import sleep
stracks = getFile("soundtracks.p")
from numpy import ceil
for cnt,style in stracks.items():
    print(cnt,'\t',style)
    maxmp = int(ceil(int(cnt.replace(',', ""))/250))
    col.downloadCollection(maxPages=min([15,maxmp]), style=["Soundtrack",style], master=True) #, fmat=fmat)

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()   

In [ ]:
def f(a_list, dbdata):
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    print(chunks)
    pool = Pool(processes=5)
    
    
    dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
    dbdata = getFile(dbname, version=3)
 
    result = pool.map_async(f, chunks)
    print(result)
 
    while not result.ready():
        print("Running...")
        time.sleep(5)
 
    return sum(result.get())
    #return sum(result.get())
    
f_amp(files[:5])

In [ ]:
a1 = "Duke Ellington"
a2 = "Coleman Hawkins"

In [ ]:
disc     = discogs()
artistDB = disc.getArtistVariationNameToIDsData()

In [ ]:
artistDB[a1]

In [ ]:
artistDB[a2]

In [ ]:
artistIDToName = disc.getArtistIDToNameData()
artistIDToRef = disc.getArtistIDToRefData()

In [ ]:
artistIDToRef['284747']

In [ ]:
artistIDToName['284747-1']

In [ ]:
from artists import artists
from artist import artist

In [ ]:
art = artist()
arts = artists(disc)

In [ ]:
ifile = '/Volumes/Music/Discog/artists/47/284747.p'
info = art.getData(ifile)

In [ ]:
len(info['Media']["Albums"])

In [ ]:
discID   = '284747'
href     = artistIDToRef[discID]
url      = arts.getArtistRef(href)
savename = arts.getArtistSavename(discID)
print(url)
url="{0}/?sort=year%2Casc&limit=500".format(url)
arts.downloadArtistURL(url, savename, force=True)

In [ ]:
html = str(getFile(ifile))
type(html)
html.find("1,221")

In [ ]:
len(html)

In [ ]:
info.keys()

for key in info.keys():
    print("\n\n\n{0}".format(key))
    val = info[key]
    if isinstance(val, dict):
        for key2,val2 in val.items():
            if isinstance(val2, list):
                print("\t{0}".format(key2))
                for key3 in val2:
                    print("\t\t{0}".format(key3))
            elif isinstance(val2, dict):
                print("\t{0}".format(key2))
                for key3,val3 in val2.items():
                    print("\t\t{0}".format(key3))
                    print("\t\t\t{0}".format(val3))
            else:
                print("\t{0}:\t{1}".format(key2,val2))
    else:
        print(info[key])

In [ ]:
from ioUtils import getFile
from fsUtils import isFile
from webUtils import getHTML, isBS4
from strUtils import fixName
from math import ceil, floor

from discogsBase import discogs

class artistIDClass:
    def __init__(self, ID=None, err=None):
        self.ID=ID
        self.err=err
            
class artistURLClass:
    def __init__(self, url=None, err=None):
        self.url = url
        self.err = err
        
class artistNameClass:
    def __init__(self, name=None, err=None):
        self.name = name
        self.err  = err

class artistMediaClass:
    def __init__(self, err=None):
        self.media = {}

class artistMediaDataClass:
    def __init__(self, album=None, url=None, aclass=None, aformat=None, artist=None, code=None, year=None, err=None):
        self.album   = album
        self.url     = url
        self.aclass  = aclass
        self.aformat = aformat
        self.artist  = artist
        self.code    = code
        self.year    = year
        self.err     = err

class artistMediaAlbumClass:
    def __init__(self, url=None, album=None, aformat=None, err=None):
        self.url     = url
        self.album   = album
        self.aformat = aformat
        self.err     = err        

class artistMediaCountsClass:
    def __init__(self, err=None):
        self.counts = {}
        self.err    = err

class artistPageClass:
    def __init__(self, ppp = None, tot = None, more=None, redo=None, err=None):
        self.ppp   = ppp
        self.tot   = tot
        if isinstance(ppp, int) and isinstance(tot, int):
            self.pages = int(ceil(tot/ppp))
        else:
            self.pages = None

        self.err   = err

        self.more  = more
        self.redo  = redo

class artistProfileClass:
    def __init__(self, profile=None, aliases=None, members=None, sites=None, groups=None, variations=None, err=None):
        self.profile    = profile
        self.aliases    = aliases
        self.members    = members
        self.sites      = sites
        self.groups     = groups
        self.variations = variations
        self.err        = err

class artistURLInfo:
    def __init__(self, name=None, url=None, ID=None, err=None):
        self.name = name
        self.url  = url
        self.ID   = ID
        self.err  = err


        
class artist(discogs):
    def __init__(self, debug=False):
        self.name  = "artist"
        self.debug = debug
        
    def getData(self, inputdata):
        if isinstance(inputdata, str):
            if isFile(inputdata):
                try:
                    bsdata = getHTML(getFile(inputdata))
                except:
                    try:
                        bsdata = getHTML(getFile(inputdata, version=2))
                    except:
                        raise ValueError("Cannot read artist file: {0}".format(inputdata))
            else:
                raise ValueError("Not sure about string input: {0}".format(inputdata))
        elif isBS4(inputdata):
            bsdata = inputdata
            pass
        else:
            raise ValueError("Not sure about input type: {0}".format(type(inputdata)))

        self.bsdata = bsdata
        
        return self.parse()
        
        
            
        
    def getNamesAndURLs(self, content):
        data = []
        for ref in content.findAll("a"):
            url    = ref.attrs['href']
            name   = ref.text

            ID = self.getArtistDiscID(url)
            data.append(artistURLInfo(name=name, url=url, ID=ID))
        return data





    #######################################################################################################################################
    ## Artist URL
    #######################################################################################################################################
    def getArtistURL(self):
        result1 = self.bsdata.find("link", {"rel": "canonical"})
        result2 = self.bsdata.find("link", {"hreflang": "en"})
        if result1 and not result2:
            result = result1
        elif result2 and not result1:
            result = result2
        elif result1 and result2:
            result = result1
        else:        
            auc = artistURLClass(err=True)
            return auc

        if result:
            url = result.attrs["href"]
            url = url.replace("https://www.discogs.com", "")
            if url.find("/artist/") == -1:
                url = None
                auc = artistURLClass(url=url, err="NoArtist")
            else:
                auc = artistURLClass(url=url)
        else:
            auc = artistURLClass(err="NoLink")

        return auc

    

    #######################################################################################################################################
    ## Artist ID
    #######################################################################################################################################                
    def getArtistDiscID(self, suburl):
        ival = "/artist"
        if isinstance(suburl, artistURLClass):
            suburl = suburl.url
        if not isinstance(suburl, str):
            aic = artistIDClass(err="NotStr")            
            return aic

        pos = suburl.find(ival)
        if pos == -1:
            aic = artistIDClass(err="NotArtist")            
            return aic

        data = suburl[pos+len(ival)+1:]
        pos  = data.find("-")
        discID = data[:pos]
        try:
            int(discID)
        except:
            aic = artistIDClass(err="NotInt")            
            return aic

        aic = artistIDClass(ID=discID)
        return aic
    
    

    #######################################################################################################################################
    ## Artist Name
    #######################################################################################################################################
    def getArtistName(self):
        result1 = self.bsdata.find("h1", {'class':'hide_desktop'})
        result2 = self.bsdata.find("h1", {'class':'hide_mobile'})
        if result1 and not result2:
            result = result1
        elif result2 and not result1:
            result = result2
        elif result1 and result2:
            result = result1
        else:        
            anc = artistNameClass(err=True)
            return anc

        if result:
            artist = result.text
            if len(artist) > 0:
                artist = fixName(artist)
                anc = artistNameClass(name=artist, err=None)
            else:
                anc = artistNameClass(name=artist, err="Fix")
        else:
            anc = artistNameClass(err="NoH1")

        return anc
    
    

    #######################################################################################################################################
    ## Artist Media
    #######################################################################################################################################
    def getArtistMediaAlbum(self, td):
        amac = artistMediaAlbumClass()
        for span in td.findAll("span"):
            attrs = span.attrs
            if attrs.get("class"):
                if 'format' in attrs["class"]:
                    albumformat = span.text
                    albumformat = albumformat.replace("(", "")
                    albumformat = albumformat.replace(")", "")
                    amac.format = albumformat
                    continue
            span.replaceWith("")

        ref = td.find("a")
        if ref:
            amac.url   = ref.attrs['href']
            amac.album = ref.text
        else:
            amac.err = "NoText"

        return amac
    
    
    def getArtistMedia(self):
        amc = artistMediaClass()
        
        table = self.bsdata.find("table", {"id": "artist"})
        if table == None:
            amc.err="NoMedia"
            return amc

        name  = None
        for tr in table.findAll("tr"):
            h3 = tr.find("h3")
            if h3:
                name = h3.text
                amc.media[name] = []
                continue


            # Album, Class, Format
            result = tr.find("td", {"class": "title"})
            album  = None
            url    = None
            albumformat = name
            if result:
                retval      = self.getArtistMediaAlbum(result)
                album       = fixName(retval.album)
                url         = retval.url
                albumformat = retval.aformat

            if album == None:
                continue

            # Code
            code = tr.attrs.get("data-object-id")

            # AlbumClass
            albumclass = tr.attrs.get("data-object-type")

            # AlbumURL
            result  = tr.find("td", {"class": "artist"})
            artists = None
            if result:
                artists = self.getNamesAndURLs(result)

            # Year
            result = tr.find("td", {"class": "year"})
            year   = None
            if result:
                year = result.text

            amdc = artistMediaDataClass(album=album, url=url, aclass=albumclass, aformat=albumformat, artist=artists, code=code, year=year)
            amc.media[name].append(amdc)
            #if debug: print "  Found album:",album,"of type:",name


        if False:
            newMedia = {}
            for name,v in media.items():
                newMedia[name] = {}
                for item in v:
                    code = item['Code']
                    del item['Code']
                    newMedia[name][code] = item

            media = newMedia

        return amc
    
    

    #######################################################################################################################################
    ## Artist Media Counts
    #######################################################################################################################################        
    def getArtistMediaCounts(self):
        amcc = artistMediaCountsClass()
        
        results = self.bsdata.findAll("ul", {"class": "facets_nav"})
        if results is None or len(results) == 0:
            amcc.err = "No Counts"
            return amcc
            
        for result in results:
            for li in result.findAll("li"):
                ref = li.find("a")
                if ref:
                    attrs = ref.attrs
                    span = ref.find("span", {"class": "facet_count"})
                    count = None
                    if span:
                        count = span.text
                        credittype    = attrs.get("data-credit-type")
                        creditsubtype = attrs.get("data-credit-subtype")
                        if credittype and creditsubtype:
                            if amcc.counts.get(credittype) == None:
                                amcc.counts[credittype] = {}
                            if amcc.counts[credittype].get(creditsubtype) == None:
                                try:
                                    amcc.counts[credittype][creditsubtype] = int(count)
                                except:
                                    amcc.counts[credittype][creditsubtype] = count
                                    amcc.err = "Non Int"

        return amcc
    
    

    #######################################################################################################################################
    ## Artist Variations
    #######################################################################################################################################
    def getArtistProfile(self):        
        result = self.bsdata.find("div", {"class": "profile"})
        data = {}
        if result:
            heads = result.findAll("div", {"class": "head"})
            heads = [x.text for x in heads]
            heads = [x.replace(":","") for x in heads]

            content = result.findAll("div", {"class": "content"})
            if len(heads) != len(content):
                raise("Mismatch in head/content")

            for i in range(len(heads)):
                if heads[i] == "Sites":
                    content[i] = self.getNamesAndURLs(content[i])
                elif heads[i] == "In Groups":
                    content[i] = self.getNamesAndURLs(content[i])
                elif heads[i] == "Variations":
                    content[i] = self.getNamesAndURLs(content[i])
                elif heads[i] == "Aliases":
                    content[i] = self.getNamesAndURLs(content[i])
                else:
                    content[i] = content[i].text
                    content[i] = content[i].strip()
                data[heads[i]] = content[i]
                
            apc = artistProfileClass(profile=data.get("Profile"), aliases=data.get("Aliases"),
                                     members=data.get("Members"), groups=data.get("In Groups"),
                                     sites=data.get("Sites"), variations=data.get("Variations"))
        else:
            apc = artistProfileClass(err="No Profile")
                
        return apc


    
    #######################################################################################################################################
    ## Artist Pages
    #######################################################################################################################################
    def getArtistPages(self):
        from numpy import ceil
        bsdata = self.bsdata


        pageData = bsdata.find("div", {"class": "pagination bottom"})
        if pageData is None:
            err = "pagination bottom"
            apc = artistPageClass(err=err)
            return apc
        else:
            x = pageData.find("strong", {"class": "pagination_total"})
            if x is None:
                err = "pagination_total"
                apc = artistPageClass(err=err)
                return apc
            else:
                txt = x.text
                txt = txt.strip()
                txt = txt.replace("\n", "")
                retval = [tmp.strip() for tmp in txt.split('of')]

                try:
                    ppp   = int(retval[0].split('–')[-1])
                    tot   = int(retval[1].replace(",", ""))
                except:
                    err   = "int"
                    apc   = artistPageClass(err=err)
                    return apc

                if ppp < 500:
                    if tot < 25:
                        apc   = artistPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistPageClass(ppp=ppp, tot=tot, redo=True, more=False)
                else:
                    if tot < 500:
                        apc   = artistPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistPageClass(ppp=ppp, tot=tot, redo=False, more=True)
                        
                return apc
            
        return artistPageClass()



    def parse(self):
        bsdata = self.bsdata
        
        retval = {}
        retval["Artist"]      = self.getArtistName()
        retval["URL"]         = self.getArtistURL()
        retval["ID"]          = self.getArtistDiscID(retval["URL"])
        retval["Pages"]       = self.getArtistPages()
        retval["Profile"]     = self.getArtistProfile()
        retval["MediaCounts"] = self.getArtistMediaCounts()
        retval["Media"]       = self.getArtistMedia()

        #print retval
        return retval    

In [ ]:
a = artist()

In [ ]:
x = a.getData(ifile)

In [ ]:
bsdata = getHTML(ifile)

In [ ]:
x['']

In [ ]:
apc.err

In [ ]:
apc.__dict__

In [ ]:
bsdata = getHTML(ifile)

In [ ]:
pages

In [ ]:
tot

In [ ]:
int(ceil(tot/ppp))

In [ ]:
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()

artistIDToRef = disc.getArtistIDToRefData()

In [ ]:

%load_ext autoreload
%autoreload

from artist import artist
from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()
toget={}
extras={}
totalSaves = 0
for i in range(19,maxModVal):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')

    saveIt = 0
    nf = len(files)
    for j,ifile in enumerate(files):
        artistID = getBaseFilename(ifile)
        if toget.get(artistID) is not None:
            continue
            
        info     = artistInfo.getData(ifile)

        ID       = info.ID.ID
            
        if artistID != ID:
            extras[ifile] = info
            continue
            
        ref = info.url.url
            
        pages  = info.pages
        if pages.redo is True:
            toget[artistID] = ref
            if len(toget) % 25 == 0:
                print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref)
                
    saveFile(idata=toget, ifile="toget3.p")

In [ ]:

%load_ext autoreload
%autoreload

from artist import artist
from artists import artists

from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()

redo={}
more={}
extras={}
totalSaves = 0
for i in range(1, 8):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')

    saveIt = 0
    nf = len(files)
    for j,ifile in enumerate(files):
        artistID = getBaseFilename(ifile)
        if redo.get(artistID) is not None:
            continue
        if more.get(artistID) is not None:
            continue
            
        info     = artistInfo.getData(ifile)

        ID       = info.ID.ID
            
        if artistID != ID:
            extras[ifile] = info
            continue
            
        ref    = info.url.url
        pages  = info.pages
        if pages.redo is True:
            redo[artistID] = ref
            print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref,'\t',pages.get())
            
        if pages.more is True:
            more[artistID] = ref
            print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref,'\t',pages.get())

                
    #saveFile(idata=toget, ifile="toget.p")

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists

for artistID,artistRef in more.items():
    ifile = arts.getArtistSavename(artistID)
    info  = art.getData(ifile)
    pages = info.pages
    
    npages = pages.pages

    print(ifile,'\t',artistRef,'\t',pages.get())

    for p in range(2, npages+1):
        url      = arts.getArtistURL(artistRef, p)
        savename = arts.getArtistSavename(artistID, p)
        if not isFile(savename):
            arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)

In [ ]:
tmp = {k: redo[k] for k in ['1028302', '1323902', '1411102', '200402', '2159702', '228202']}
tmp

In [ ]:
redo = tmp.copy()
redo

In [ ]:
redo

In [ ]:
%load_ext autoreload
%autoreload

redo={"780701": "/artist/780701-Whitfield-Strong"}

from artist import artist
from artists import artists

from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()
arts = artists(disc)

from time import sleep
for artistID,artistRef in redo.items():
    url = arts.getArtistURL(artistRef)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, force=True)
    sleep(1)